In [1]:
import requests 
from pandas import DataFrame
from collections import OrderedDict
from selenium import webdriver
from BeautifulSoup import BeautifulSoup
from time import sleep
# http://www.nacao.org.cn/portal/publish/index.html
name = u'连云港玖酬商务信息咨询有限公司'
mainUrl = 'http://www.nacao.org.cn/portal/publish/index.html'

In [2]:
driver = webdriver.Firefox()
driver.maximize_window()

In [3]:
def switchWindow(n):
    """ 切换窗口 """
    driver.switch_to.window(driver.window_handles[n])

In [4]:
def closeOthers():
    """ 关闭其他窗口 """
    while len(driver.window_handles) > 1:
        switchWindow(1)
        driver.close()

In [5]:
def waitCode():
    """ 等待输入验证码 """
    switchWindow(1)
    # 等待页面加载完毕
    while True:
        soup = BeautifulSoup(driver.page_source)
        if u'数据加载中请稍候...' in unicode(soup):
            break
        sleep(1)
    
    # 等待用户输入验证码
    while True:
        soup = BeautifulSoup(driver.page_source)
        if u'数据加载中请稍候...' not in unicode(soup):
            break
        sleep(1)

In [6]:
def readDataFromUrl(url):
    respnose = requests.get(url)
    soup = BeautifulSoup(respnose.content)
    data = OrderedDict()
    data[u'统一社会信用代码'] = soup.find('h3').text.split(u'：')[1].strip()
    liList = soup.findAll('li', {'class': 'cnt'})
    data[u'组织机构名称'] = liList[0].text
    data[u'机构注册地址'] = liList[1].text
    data[u'成立日期'] = liList[2].text
    data[u'经营期限'] = liList[3].text
    return data

In [7]:
def getResultSet():
    switchWindow(1)
    soup = BeautifulSoup(driver.page_source)
    aList = filter(lambda x: x.get('href').startswith('/gaiwan_shehui.jsp'), soup.findAll('a'))
    urlList = map(lambda x: 'https://s.nacao.org.cn/' + x.get('href'), aList)
    dataList = map(readDataFromUrl, urlList)
    if len(dataList) == 0:
        return None
    else:
        return DataFrame(dataList)

In [8]:
def fetch(name):
    # 切换到主窗口
    closeOthers()
    switchWindow(0)

    # 输入查询关键字
    if driver.current_url != mainUrl:
        driver.get(mainUrl)    
    # 这里可能由于版本问题, send_keys无效
    driver.execute_script(u"document.getElementById('tit2').value = '%s'" % name)
    driver.execute_script('submitForm(2)')

    # 切换查询窗口, 并等待验证码
    switchWindow(1)
    waitCode()
    df = getResultSet()
    
    return df

In [9]:
for i in range(3):
    fetch(u'青海版筑建筑劳务有限公司')

In [10]:
fetch(u'青海版筑建筑劳务有限公司')

,统一社会信用代码,组织机构名称,机构注册地址,成立日期,经营期限
0,91630103MA757E0M05,青海版筑建筑劳务有限公司,西宁市城中区滨河南路78号4号楼3单元3265室,2017-08-15,自2017年08月15日 至 2037年08月14日
